In [1]:
import os;
import locale;
import sys;

from collections import defaultdict

import pandas as pd
import matplotlib.pyplot as plt
import requests

#from pattern import web

from bs4 import BeautifulSoup
from bs4 import UnicodeDammit
%matplotlib inline
import time 
import random
from tqdm import tqdm_notebook as tqdm
from line_profiler import LineProfiler



In [2]:
HEADER = {'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/51.0.2704.79 Chrome/51.0.2704.79 Safari/537.36'}
#PROXY = {"https": "http': 'http://10.10.1.10:3128"}
PROXY = {"https": "http': 'http:/178.88.119.150"}

# Parse soup to get all the state's url
def get_state_urls(soup):
    state_urls = []
    states = []
    for col in soup.find_all("ul", class_='tab-list tab-list-short'):
        for state in col.find_all('a'):
            state_urls.append(state.get('href')) 
            states.append(state.text)
    state_urls.remove('http://www.city-data.com/smallTowns.html')
    states.remove('small')
    return state_urls, states

# Request html with proxy and headers in mind
def request_soup(link):
    website_html = requests.get(link, proxies = PROXY, headers = HEADER)
    soup = BeautifulSoup(website_html.text, 'html.parser')
    return soup

# Parse soup to get all the city's url
def get_cities_url(city_urls, cities, state_soup):
    
    table = state_soup.find("table", {'class' : "tabBlue tblsort tblsticky"}).tbody
    state_data = table.find_all("tr", class_ = "rB")
    
    for row in state_data:
        city_urls.append('http://www.city-data.com/city/' + row.find('a').get('href'))
        cities.extend(row.find('a').contents)
        #cur_population.append(row.find_all('td')[-1].text)
    
    return city_urls, cities

# Parse soup to get data of a city
def parse_city(features_dict, city_soup, city, state):
    
    attributes = []
    values = []
    
    # Adding city name
    attributes.append("City:")
    values.append(city)

    # Adding recorded population
    attributes.append("Population" if city_soup.find("section", class_ = "city-population") else "")
    values.append(city_soup.find("section", class_ = "city-population").find('b').next_sibling if city_soup.find("section", class_ = "city-population") else "")

    # Adding population by sex
    attributes.extend([a.text for a in city_soup.find("section", class_ = "population-by-sex").find_all('b')])
    values.extend([v.next_sibling.string.replace('\xa0',"") for v in city_soup.find("section", class_ = "population-by-sex").find_all('b')])

    # Adding population by sex
    attributes.append("Males:" if city_soup.find("section", class_ = "median-age") else "")
    attributes.append("Females:" if city_soup.find("section", class_ = "median-age") else "")
    tmp_data = [v.text.replace('\xa0',"") for v in city_soup.find("section", class_ = "median-age").find_all('td')]
    values.append(tmp_data[1])
    values.append(tmp_data[3])

    # Adding median income
    attributes.append("Median Income" if city_soup.find("section", class_ = "median-income") else "")
    values.append(city_soup.find("section", class_ = "median-income").find('b').next_sibling.string.replace("(","") if city_soup.find("section", class_ = "median-income") else "")

    attributes.append("Estimated median house or condo value in 2016:")
    values.append(city_soup.find("section", class_ = "median-income").find("b", string = "Estimated median house or condo value in 2016:").next_sibling.replace('(',"") if city_soup.find("section", class_ = "median-income") else "")

    attributes.append("Estimated median house or condo value in 2016:")
    values.append(city_soup.find("section", class_ = "median-income").find("b", string = "Estimated median house or condo value in 2016:").next_sibling.replace('(',"") if city_soup.find("section", class_ = "median-income") else "")

    # Adding elevation
    attributes.append("Elevation" if city_soup.find("section", class_ = "elevation") else "")
    values.append(city_soup.find("section", class_ = "elevation").find('b').next_sibling if city_soup.find("section", class_ = "elevation") else "")

    # Adding population density
    attributes.append(city_soup.find("section", class_ = "population-density").find('b').text if city_soup.find("section", class_ = "population-density") else "")
    values.append(city_soup.find("section", class_ = "population-density").find('b').next_sibling if city_soup.find("section", class_ = "population-density") else "")

    attributes.append(city_soup.find("section", class_ = "population-density").find_all('b')[2].text if city_soup.find("section", class_ = "population-density") else "")
    values.append(city_soup.find("section", class_ = "population-density").find_all('b')[2].next_sibling if city_soup.find("section", class_ = "population-density") else "")

    # Adding cost of living
    attributes.append("Mar. 2016 cost of living index: " if city_soup.find("section", class_ = "cost-of-living-index") else "")
    values.append(city_soup.find("section", class_ = "cost-of-living-index").find('b').next_sibling if city_soup.find("section", class_ = "cost-of-living-index") else "")

    features_dict = dict(zip(attributes,values))

    return features_dict


In [8]:
##### MAIN ######

def _init():
    HEADER = {"User-Agent": "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.2.8) Gecko/20100722 Firefox/3.6.8 GTB7.1 (.NET CLR 3.5.30729)"}
    #PROXY = {"https": "https//59.110.7.190:1080"}
    PROXY = {"https": "https//178.88.119.150"}
    #url = "http://www.city-data.com/"
    url = 'http://www.city-data.com/indexes/cities/?fbclid=IwAR1LvY4RZUePzl9FkAP8Yv6W88cwA81G6gFFPLCx-xXFv5El0lBR6UHFPFo'
    website_html = requests.get(url, proxies = PROXY, headers = HEADER, verify = False)
    soup = BeautifulSoup(website_html.text, 'html.parser')

    state_urls, states = get_state_urls(soup)

    return state_urls, states

state_urls, states = _init()

In [9]:
print(states)

['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']


In [12]:
all_cities_feature_dicts = []
state_feature_dicts = []
def scrape(state_urls, states):
    for state_idx in tqdm(range(2)):
        state_feature_dicts = []
        state_soup = request_soup(state_urls[state_idx])
        time.sleep(random.randint(2,6)) # in seconds 

        city_urls, cities = get_cities_url([], [], state_soup)

        for city_idx in tqdm(range(len(cities))):
            print('Parsing for city: ', city_urls[city_idx])
            city_soup = request_soup(city_urls[city_idx])
            time.sleep(random.randint(1,10)) # in seconds 
            feature_dict = parse_city({},
                                      city_soup,
                                      cities[city_idx],
                                      states[state_idx])
            if "" in feature_dict: del feature_dict[""]
            all_cities_feature_dicts.append(feature_dict)
            state_feature_dicts.append(feature_dict)
        df = pd.DataFrame.from_dict(state_feature_dicts)
        df.to_csv(states[state_idx] + ".csv")
        
    return all_cities_feature_dicts

In [13]:
all_city_feature_dicts = scrape(state_urls[49:], states[49:])

Parsing for city:  http://www.city-data.com/city/Beckley-West-Virginia.html
Parsing for city:  http://www.city-data.com/city/Bluefield-West-Virginia.html
Parsing for city:  http://www.city-data.com/city/Bridgeport-West-Virginia.html
Parsing for city:  http://www.city-data.com/city/Charleston-West-Virginia.html
Parsing for city:  http://www.city-data.com/city/Cheat-Lake-West-Virginia.html
Parsing for city:  http://www.city-data.com/city/Clarksburg-West-Virginia.html
Parsing for city:  http://www.city-data.com/city/Cross-Lanes-West-Virginia.html
Parsing for city:  http://www.city-data.com/city/Dunbar-West-Virginia.html
Parsing for city:  http://www.city-data.com/city/Elkins-West-Virginia.html
Parsing for city:  http://www.city-data.com/city/Fairmont-West-Virginia.html
Parsing for city:  http://www.city-data.com/city/Huntington-West-Virginia.html
Parsing for city:  http://www.city-data.com/city/Hurricane-West-Virginia.html
Parsing for city:  http://www.city-data.com/city/Kabletown-distric

Parsing for city:  http://www.city-data.com/city/Bridger-Valley-Wyoming.html
Parsing for city:  http://www.city-data.com/city/Casper-Wyoming.html
Parsing for city:  http://www.city-data.com/city/Cheyenne-Wyoming.html
Parsing for city:  http://www.city-data.com/city/Cody-Wyoming.html
Parsing for city:  http://www.city-data.com/city/Douglas-Wyoming.html
Parsing for city:  http://www.city-data.com/city/Evanston-Wyoming.html
Parsing for city:  http://www.city-data.com/city/Gillette-Wyoming.html
Parsing for city:  http://www.city-data.com/city/Green-River-Wyoming.html
Parsing for city:  http://www.city-data.com/city/Jackson-Hole-Wyoming.html
Parsing for city:  http://www.city-data.com/city/Jackson-Wyoming.html
Parsing for city:  http://www.city-data.com/city/Lander-Wyoming.html
Parsing for city:  http://www.city-data.com/city/Laramie-Wyoming.html
Parsing for city:  http://www.city-data.com/city/Powell-Wyoming.html
Parsing for city:  http://www.city-data.com/city/Rawlins-Wyoming.html
Parsing

### Code Profiling

In [64]:
def test_one_state():
    city_feature_list = []
    feature_dict = scrape(state_urls, states)
    city_feature_list.append(feature_dict)
    time.sleep(random.randint(1,10))

In [65]:
lp = LineProfiler()
lp_wrapper =lp(test_one_state)

In [66]:
lp.add_function(scrape)

In [69]:
result = lp_wrapper()

Parsing for city:  http://www.city-data.com/city/Anchorage-Alaska.html
Parsing for city:  http://www.city-data.com/city/Badger-Alaska.html
Parsing for city:  http://www.city-data.com/city/Bethel-Alaska.html
Parsing for city:  http://www.city-data.com/city/College-Alaska.html
Parsing for city:  http://www.city-data.com/city/Fairbanks-Alaska.html
Parsing for city:  http://www.city-data.com/city/Juneau-Alaska.html
Parsing for city:  http://www.city-data.com/city/Kalifornsky-Alaska.html
Parsing for city:  http://www.city-data.com/city/Kenai-Alaska.html
Parsing for city:  http://www.city-data.com/city/Ketchikan-Alaska.html
Parsing for city:  http://www.city-data.com/city/Knik-Fairview-Alaska.html
Parsing for city:  http://www.city-data.com/city/Kodiak-Alaska.html
Parsing for city:  http://www.city-data.com/city/Lakes-Alaska.html
Parsing for city:  http://www.city-data.com/city/Meadow-Lakes-Alaska.html
Parsing for city:  http://www.city-data.com/city/Palmer-Alaska.html
Parsing for city:  htt

In [70]:
lp.print_stats()

Timer unit: 1e-06 s

Total time: 107.629 s
File: <ipython-input-63-19f331831764>
Function: scrape at line 3

Line #      Hits         Time  Per Hit   % Time  Line Contents
     3                                           def scrape(state_urls, states):
     4         2          5.0      2.5      0.0      for state_idx in range(1): 
     5         1    1532152.0 1532152.0      1.4          state_soup = request_soup(state_urls[state_idx])
     6         1    2002113.0 2002113.0      1.9          time.sleep(random.randint(2,6)) # in seconds 
     7                                           
     8         1      54959.0  54959.0      0.1          city_urls, cities = get_cities_url([], [], state_soup)
     9                                           
    10        19         14.0      0.7      0.0          for city_idx in range(len(cities)):
    11        18       2594.0    144.1      0.0              print('Parsing for city: ', city_urls[city_idx])
    12        18   28921165.0 1606731.4 

In [14]:
data = pd.read_csv("GA.csv")
data

,Unnamed: 0,City:,Elevation,Estimated median house or condo value in 2016:,Females:,Land area:,Males:,Mar. 2016 cost of living index:,Median Income,Population,Population density:
0,0,"Acworth, GA",913 feet,"$179,902",36.5 years,7.08,35.3 years,95.9,"$56,935","21,867 (100% urban, 0% rural).","3,090"
1,1,"Albany, GA",208 feet,"$88,800",36.5 years,55.50,33.9 years,88.5,"$34,565","74,904","1,349"
2,2,Alpharetta,1137 feet,"$408,600",36.5 years,21.40,38.7 years,97.7,"$95,919","65,334","3,058"
3,3,"Americus, GA",420 feet,"$93,360",36.5 years,10.50,29.3 years,81.3,"$26,112","16,283 (98% urban, 2% rural).","1,553"
4,4,Appling,295 feet,"$157,248",33.4 years,140.00,37.1 years,85.0,"$62,641","8,141.",58
5,5,Armuchee,643 feet,"$136,256",33.4 years,95.60,35.3 years,85.2,"$54,779","9,817.",103
6,6,"Athens-Clarke County, GA",NaN,"$170,900",36.5 years,117.80,27.4 years,93.4,"$34,592","123,456","1,048"
7,7,"Athens, GA",758 feet,"$178,040",33.4 years,64.40,24.2 years,101.7,"$30,753","85,116.","1,321"
8,8,"Atlanta, GA",1050 feet,"$262,600",36.5 years,131.70,33.0 years,98.7,"$53,843","472,506","3,586"
9,9,"Auburn, GA",1050 feet,"$124,975",36.5 years,5.34,32.8 years,94.3,"$54,381","7,254 (94% urban, 6% rural).","1,359"


In [96]:
print(city_feature_dicts)

[{'City:': 'Anchorage, AK', 'Population': ' 298,192', 'Males:': '33.0 years', 'Females:': '33.5 years', 'Median Income': ' $85,634 ', 'Estimated median house or condo value in 2016:': ' $320,800 ', 'Elevation': ' 101 feet', 'Land area:': ' 1697.2 ', 'Population density:': ' 176 ', 'Mar. 2016 cost of living index: ': ' 102.6 '}, {'City:': 'Badger, AK', 'Population': ' 19,482.', 'Males:': '31.9 years', 'Females:': '33.5 years', 'Median Income': ' $77,328', 'Estimated median house or condo value in 2016:': ' $226,900\r\n', 'Mar. 2016 cost of living index: ': ' 104.2 '}, {'City:': 'Bethel, AK', 'Population': ' 6,415 (74% urban, 26% rural). ', 'Males:': '29.1 years', 'Females:': '33.5 years', 'Median Income': ' $79,712 ', 'Estimated median house or condo value in 2016:': ' $265,717 ', 'Land area:': ' 43.8 ', 'Population density:': ' 147 ', 'Mar. 2016 cost of living index: ': ' 91.9 '}, {'City:': 'College', 'Population': ' 12,964. ', 'Males:': '29.3 years', 'Females:': '33.5 years', 'Median 